Elegir un sector de interés (“Turismo”, “Seguridad”, “Incendios”, etc.) y un conjunto de medios (medios de Los Ríos o de Los Lagos).
Extraer un conjunto de noticias a partir de Sophia Search, definiendo palabras claves y rango de fechas.
Diagnósticar la calidad del dataset obtenido. Filtrar, limpiar y mejorar la calidad del dataset.
Definir algunas preguntas de análisis cuantitativa y cualitativa
Procesar los datos para responder a las preguntas de análisis
Generar indicadores estádisticos o visualizaciones de datos para presentar los resultados


Discusión (30 minutos)

¿Qué métodos de análisis de datos se podría utilizar?
¿Qué variables son relevantes considerar?
¿Qué tipo de visualización de datos o indicadores estádisticos?


In [1]:
import elasticsearch
from datetime import datetime
import pandas as pd
from pandasql import sqldf

In [3]:
from tqdm import tqdm # Barra de progreso
from collections import Counter

#Pandas
import pandas as pd
from pandasql import sqldf

#Spacy
import spacy
from spacy.matcher import PhraseMatcher
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_md")

#Pandas, Numpy, Matplotlib, Seaborn
import geopandas as gpd
import shapely as shp
import pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.ticker as ticker
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

In [4]:
TOPIC="incendios forestales"
PALABRAS_CLAVES = [
    "incendio",
"prevención",
"combate",
"sequía",
"riesgo",
"alerta",
"extinción",
"evacuación",
"seguridad",
"rescate",
"bomberos",
"quema controlada",
"alto riesgo",
"ambiental",
"emergencia",
"monitoreo",
"cambio climático",
"rehabilitación",
"forestal",
"CONAF",
"bosque",
"peligro",
"protección",
"equipo",
"coordinación",
"mitigación",
"planificación",
"información",
"capacitación",
"gestión",
"intervención",
"daño",
"prevención de incendios",
"recursos",
"apoyo",
"control",
"respuesta",
"estrategia",
"rehabilitación ambiental"
]


In [5]:
CITIES = ["Valdivia", "Mariquina", "Lanco", "Máfil", "Corral",
          "Paillaco", "Panguipulli", "La Unión", "Río Bueno", "Futrono",
         "Los Lagos", "Lago Ranco"]

In [6]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic_reader"
PASS= "1QpbX5Yd"

In [7]:
MEDIA_OUTLETS=["rioenlinea","diariopaillaco","diariomafil",
               "diariolaguino","diariolaunion","diariofutrono",
              "diariolagoranco","elnavegante","lapailapaillaco",
              "centralnoticiaspanguipulli","diariodevaldivia",
              "lavozdepaillaco","diariolanco","suractual",
              "noticiaslosrios","diarioriobueno","diariosanjose","diariolaguino",
              "diariocorral"]
COUNTRY="chile"
KEYWORD="incendio forestal"
SIMPLE_KEYWORD=True #esta variable es 'True' si KEYWORD es vacio o contiene una palabra simple, y 'False' si contiene una expresion o un concepto compuesto

In [8]:
def sophia_search(from_date,to_date,df_list):

    es = elasticsearch.Elasticsearch(
        IP,
        #port=PORT,
        http_auth=(USER, PASS)
    )

    match=""
    if (SIMPLE_KEYWORD):
        match="match"
    else:
        match="match_phrase"

    if len(KEYWORD)==0:
        query = { 
        "bool": { 
        "filter": [
            {"range": {
          "date": {
            "gte": from_date,
            "lt": to_date
          }}},
            { "term":  { "country": COUNTRY }},
            { "terms":  { "media_outlet": MEDIA_OUTLETS }} 
        ]
        }  
    }
    
    else:
        query = { 
        "bool": { 
        "must": [
            {match: { "text":KEYWORD}}
        ],
        "filter": [
            {"range": {
          "date": {
            "gte": from_date,
            "lt": to_date
          }}},
            { "term":  { "country": COUNTRY }},
            { "terms":  { "media_outlet": MEDIA_OUTLETS }} 
        ]
        }  
    }
        
    res = es.search(index="news", query=query, size=10000)
    print("Fechas: " + from_date + "|" + to_date)
    print("Son %d noticias encontradas..." % res['hits']['total']['value'])
    
    import pandas as pd
    data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

    df = pd.DataFrame(data)  

    for hit in res['hits']['hits']:
        id_news = hit['_source']['id_news']
        country = hit['_source']['country']
        media_outlet = hit['_source']['media_outlet']
        url = hit['_source']['url']
        title = hit['_source']['title']
        text = hit['_source']['text']
        date = hit['_source']['date']
        search = KEYWORD

        new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}

        df = df.append(new_row, ignore_index=True)
    df_list.append(df)

In [10]:
#Guardar noticias por fecha meses de 2018 a 2022
df_list = []
months = [
    ["2018-01-01", "2018-01-31"],
    ["2018-02-01", "2018-02-28"],
    ["2018-03-01", "2018-03-31"],
    ["2018-04-01", "2018-04-30"],
    ["2018-05-01", "2018-05-31"],
    ["2018-06-01", "2018-06-30"],
    ["2018-07-01", "2018-07-31"],
    ["2018-08-01", "2018-08-31"],
    ["2018-09-01", "2018-09-30"],
    ["2018-10-01", "2018-10-31"],
    ["2018-11-01", "2018-11-30"],
    ["2018-12-01", "2018-12-31"],
    ["2019-01-01", "2019-01-31"],
    ["2019-02-01", "2019-02-28"],
    ["2019-03-01", "2019-03-31"],
    ["2019-04-01", "2019-04-30"],
    ["2019-05-01", "2019-05-31"],
    ["2019-06-01", "2019-06-30"],
    ["2019-07-01", "2019-07-31"],
    ["2019-08-01", "2019-08-31"],
    ["2019-09-01", "2019-09-30"],
    ["2019-10-01", "2019-10-31"],
    ["2019-11-01", "2019-11-30"],
    ["2019-12-01", "2019-12-31"],
    ["2020-01-01", "2020-01-31"],
    ["2020-02-01", "2020-02-29"],
    ["2020-03-01", "2020-03-31"],
    ["2020-04-01", "2020-04-30"],
    ["2020-05-01", "2020-05-31"],
    ["2020-06-01", "2020-06-30"],
    ["2020-07-01", "2020-07-31"],
    ["2020-08-01", "2020-08-31"],
    ["2020-09-01", "2020-09-30"],
    ["2020-10-01", "2020-10-31"],
    ["2020-11-01", "2020-11-30"],
    ["2020-12-01", "2020-12-31"],
    ["2021-01-01", "2021-01-31"],
    ["2021-02-01", "2021-02-28"],
    ["2021-03-01", "2021-03-31"],
    ["2021-04-01", "2021-04-30"],
    ["2021-05-01", "2021-05-31"],
    ["2021-06-01", "2021-06-30"],
    ["2021-07-01", "2021-07-31"],
    ["2021-08-01", "2021-08-31"],
    ["2021-09-01", "2021-09-30"],
    ["2021-10-01", "2021-10-31"],
    ["2021-11-01", "2021-11-30"],
    ["2021-12-01", "2021-12-31"],
    ["2022-01-01", "2022-01-31"],
    ["2022-02-01", "2022-02-28"],
    ["2022-03-01", "2022-03-31"],
    ["2022-04-01", "2022-04-30"],
    ["2022-05-01", "2022-05-31"],
    ["2022-06-01", "2022-06-30"],
    ["2022-07-01", "2022-07-31"],
    ["2022-08-01", "2022-08-31"],
    ["2022-09-01", "2022-09-30"],
    ["2022-10-01", "2022-10-31"],
    ["2022-11-01", "2022-11-30"],
    ["2022-12-01", "2022-12-31"],
    ["2023-01-01", "2023-01-31"],
    ["2023-02-01", "2023-02-28"],
    ["2023-03-01", "2023-03-31"],
    ["2023-04-01", "2023-04-30"],
    ["2023-05-01", "2023-05-31"]
]

for month in months:
    sophia_search(month[0],month[1],df_list)

Fechas: 2018-01-01|2018-01-31
Son 19 noticias encontradas...


C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\L

Fechas: 2018-02-01|2018-02-28
Son 10 noticias encontradas...


C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\L

Fechas: 2018-03-01|2018-03-31
Son 10 noticias encontradas...


...

Fechas: 2023-03-01|2023-03-31
Son 93 noticias encontradas...


C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\L

Fechas: 2023-04-01|2023-04-30
Son 29 noticias encontradas...


C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_5124\848799684.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\sebas\AppData\L

Fechas: 2023-05-01|2023-05-31
Son 0 noticias encontradas...


In [11]:
#agrupar los dataframes mensuales en uno solo
df_periodo = pd.concat(df_list)
print(len(df_periodo))

3175


In [12]:
#Eliminar noticias duplicadas que tienen la misma URL
df_periodo = df_periodo.drop_duplicates(subset='url', keep='first')
print(len(df_periodo))

3163


In [13]:
#guardar en un CSV
df_periodo.to_csv("noticias_incendios_losrios_20182023.csv")

In [15]:
data = pd.read_csv('noticias_incendios_losrios_20182023.csv')
data

,Unnamed: 0,id_news,country,media_outlet,url,title,text,date,search
0,0,47714896.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Bomberos de Paillaco incorporó un dron en la e...,Casi tres horas debieron trabajar bomberos de ...,2018-01-13,incendio forestal
1,1,47538677.0,chile,suractual,https://www.suractual.cl/2018/01/28/situacion-...,Situación de incendios forestales en la Regió...,"La siguiente es una actualización, a las 17:30...",2018-01-28,incendio forestal
2,2,47572503.0,chile,suractual,https://www.suractual.cl/2018/01/19/nueva-fami...,Nueva familia de huemules registran cámaras i...,Nuevamente el uso de cámaras trampa en la Rese...,2018-01-19,incendio forestal
3,3,47538671.0,chile,suractual,https://www.suractual.cl/2018/01/30/autoridade...,Autoridades realizaron balance de incendios fo...,Menor número de ocurrencia y de superficie afe...,2018-01-30,incendio forestal
4,4,49543061.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/incendio-forest...,Incendio forestal en sector el Maitén es el se...,Este jueves por la tarde se desencadeno un inc...,2018-01-19,incendio forestal
...,...,...,...,...,...,...,...,...,...
3158,24,49431622.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Inauguran planta de pellet en La Unión: espera...,"En la ruta T-70 camino a la Unión, Km 4,5, com...",2023-04-12,incendio forestal
3159,25,49431881.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Esperan fallo de la Corte Suprema por caso Car...,"El jueves 30 de marzo, en la Corte Suprema se ...",2023-04-01,incendio forestal
3160,26,49434082.0,chile,diariodevaldivia,https://www.diariodevaldivia.cl/noticia/actual...,Esperan fallo de la Corte Suprema por caso Car...,"El jueves 30 de marzo, en la Corte Suprema se ...",2023-04-01,incendio forestal
3161,27,49433494.0,chile,lavozdepaillaco,https://www.lavozdepaillaco.cl/paillaco-realiz...,Paillaco: Realizaron Plaza Ciudadana en Itrop...,Una nueva Plaza Ciudadana del Gobierno en Terr...,2023-04-09,incendio forestal


### Buscar comunas en las noticias

In [1]:
matcher_cities = PhraseMatcher(nlp.vocab)

for city in CITIES:
    matcher_cities.add(city, [nlp(city)])

NameError: name 'PhraseMatcher' is not defined

In [ ]:
for index,row in df.iterrows():
    print(index)
    txt = str(row["title"])+" "+str(row["text"])
    
    try:
        doc = nlp(txt)
        matches_cities = matcher_cities(doc)

        for match_id, start, end in matches_cities:
            span = doc[start:end]  # The matched span
            df.at[index,span.text.replace(" ","_")]=1
    
    except Exception as e:
        print(e)
        pass

In [ ]:
df.to_csv("noticias_incendios_losrios_porcomuna.csv")
df